## Problem Set 6: Turing Machines

_csc427, semester 222
<br>
university of miami
<br>
date: 21 mar 2022_


---

### Student name: Diep Vu

---

## Turing Machines


This problem set is exercises in the programming of a Turing Machine. The Turing Machine
simulator has been written. Because TM programs can be large, we will not load hand-written
data structures into the simulator, as we did with the Finite Automata simulator. 
Rather, a parser program has been written, a sort of compiler for Turing machines.

The parser will take a string that describes the TM and create the data structures for us.
The syntax of this "Turing Machine programming language" is given below. 

The code for the simulator is in the file turing_machine_sim.py, found in the class folder and 
must be copied to your project folder. The classes and defines are made available with the 
import statement shown in the following code box. You must run this box
to import the contents of the file.

__Note:__ If for some reason you change the .py file, _you must
run this import statement again_. The Jupyter system does not monitor for changes in imported
files, and your changes will not be active in this notebook until you re-import.
It is always advisable to "Restart and Run All" from time to time, to make sure your data
environment is fresh.

In [37]:
from turing_machine_sim import *


## a helper function. You will need this to run your basic tests

def run_exercise(exercise_list,verbose='none'):
    count =  0
    for (label, description, test_vector) in exercise_list:
        if create_and_test_turing_machine(label, description, test_vector, verbose):
            count += 1
    print(f'\ncorrect: {count} out of {len(exercise_list)} machines tested')
    return count==len(exercise_list)



### Programs must halt.

Your programs must halt to be correct. There is no general way to test for future halting of a program that has not yet halted. Therefore the only option I have is to terminate a program after a reasonable amount of computation steps. The amount of computation steps permitted depends on the length of the input:
the longer the input, the more steps permitted.

A program that is still running after the premiitted number of steps, will be terminated and the
the result considered incorrect.

For those interested in the details, the computer_tm method returns True or False, and the property "result"
is set to additional information about the computation. The class property "result_reasons" gives the possible
values for result. The result property will be checked to see if the machine was force halted.


### Syntax 


The file is a sequence of stanzas. Each stanza starts with a head line starting 
in the first column. The first word in the head line as a keyword indicating 
the type of stanza. A stanza may have continuation lines, each are indented.

The # character starts a comment, and the comment extends to the end of the line. 
Since the # is used for this 
purpose it cannot be a tape symbol. If you wish to code programs in the textbook that use
the # symbol, substitute with one of the allowed punctuations.

The syntax can be roughly described by the following grammar.

- __Stanza__ -> __StartStanza__ | __AcceptStanza__ | __RejectStanza__ | __StateStanza__
- __StartStanza__ -> "start" ":" __Ident__
- __AcceptStanza__ -> "accept" ":" __Ident__ (\n\t __Ident__)*
- __RejectStanza__ -> "reject" ":" __Ident__ (\n\t __Ident__)*
- __StateStanza__ -> "state" ":" __Ident__ (\n\t __StateTransition__)+
- __StateTransition__ -> (__Symbol__|__Special__) (__Symbol__|__Special__) __Action__ __Ident__
- __Symbol__ -> tape symbols are alphanumeric or punctuation ! $ % & ( ) * + , - . or /
- __Special__ -> the : and _
- __Action__ -> the characters l, r and n or uppercase L, R and N.
- __Ident__ -> a nonempty string of alphanumerics

There must be exactly one start, accept and reject stanzas.

The underscore (_) substitutes for the blank. It is the default chacters of all unfilled cells on the tape.
Using the blank for the initial tape contents is allowed, it will be rewritten to the underscore character.

An Ident is the label of a state.

A StateStanza names to from state after the colon and each StateTransition line gives
one transition according to the syntax: 
<pre>    read-symbol write-symbol action new-state</pre>
The action is either move left, move rigth, or no move. If the code for the action, l, r, or n, 
is captialized, the machine configuration is printed after the transition. A debugging feature.

The Special colon (:) occurs in a transition rule is a wildcard match,
- as the read-symbol, matches anything;
- as the write-symbol, is equal the the actual read-symbol (write what you read);
- a wildcard match has a lower precedence than a match to a specific symbol.


A missing transition halts with reject. This and the wildcard are convenience features to
shorten the TM programs.


### Example: Recognize a Simple RE

An example for the syntax is the following program recognizing a+b+c+. 

Note the call to compute_tm requires a string, a parameter giving maximum number of steps
permitted to the computation, and an optional verbose parameter.

In [5]:
tm_abc= """# a TM program to recognize a+b+c+

accept: A
reject: R
start: q0

state: q0    
    a a r q0  # should start with an a, and loop over further a's
    b b r q1  # until a b.
              # note: missing transtions cause the machine to reject (c or _)
    
state: q1
    b b r q1  # continue over further b's
    c c r q2  # until a c.
    
state: q2
    c c r q2  # continue over further c's
    _ _ r A   # until the end of the tape

"""

# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

tm = MachineParser.create_from_description(tm_abc)
tm.compute_tm('aabbcc', max_steps, verbose=verbose)
tm.compute_tm('ca', max_steps, verbose=verbose)



0 [q0]	[a]abbcc_
1 [q0]	a[a]bbcc_
2 [q0]	aa[b]bcc_
3 [q1]	aab[b]cc_
4 [q1]	aabb[c]c_
5 [q2]	aabbc[c]_
6 [q2]	aabbcc[_]
7 [A]	aabbcc_[_]
accept (ok)
0 [q0]	[c]a_
reject (transition missing)


False

### Example: Erase the Tape

Accept everything, while erasing the tape leaving the head at the start of the tape.

A Turing Machine opens up another aspect of computation. So far, the algorithms in this
class have only a boolean result: true or false for membership of an element in a set.
(In fact, membership in a subset of a set, as we always assume the input is of proper form.)

This example intrduces another meaning for computation. It is the more common meaning of
"doing something" &mdash; in this case, the TM erases the tape.

In [19]:
tm_erase = """# erase the tape

accept: A
reject: R
start: q0

state: q0    # write a tape endmarker
    : $ R q1
    
state: q1
    : : r q1 # more right not replacing symbols
    _ _ l q2 # begin the erase while returning
    
state: q2
    : _ l q2 # erase symbols
    $ _ N A  # stop at the endmarker
"""


# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

tm = MachineParser.create_from_description(tm_erase)
tm.compute_tm('abc', max_steps, verbose=verbose)


0 [q0]	[&]_
1 [q1]	$[_]
2 [q2]	[$]_
3 [A]	[_]_
accept (ok)


True

In [60]:
tm_3_8a = """# Exercise 3.8(a)
# the string w contains twice as many 0s as 1s

accept: A
reject: R
start: q0
    
state: q0
    0 y r q1
    1 y r q2
    
state: q1
    0 0 r q1
    x x r q1
    1 x l q5
    
state: q2
    1 1 r q2
    x x r q2
    0 x l q5
    
state: q3
    0 0 l q3
    1 1 l q3
    x x l q3
    y y r q4
    
state: q4
    x x r q4
    _ _ r A
    0 x r q1
    1 x r q2
    
state: q5
    x x r q5
    1 1 r q5
    0 x l q3
    y y r q5
    

"""

verbose = 'explain'

max_steps = 100

tm = MachineParser.create_from_description(tm_3_8a)
tm.compute_tm('010', max_steps, verbose=verbose)

0 [q0]	[0]10_
1 [q1]	y[1]0_
2 [q5]	[y]x0_
3 [q5]	y[x]0_
4 [q5]	yx[0]_
5 [q3]	y[x]x_
6 [q3]	[y]xx_
7 [q4]	y[x]x_
8 [q4]	yx[x]_
9 [q4]	yxx[_]
10 [A]	yxx_[_]
accept (ok)


True

### Exercise A

Program the machines M1, M2 and M3 from the class text.

Note that M3 is a decision problem for what might otherwise be thought of as a computation. 
Generaly "to multiply" means to produce the correct output associated with the presented inputs.
You are given 3 and 2 and respond with a 6. 

M3 casts this as a decision problem: given i, j and k, is k == i &times; j.

In [61]:
# Turing Machine M1, Sipser 3ird ed page 173, Sipser 2nd ed page 145
tm_M1 = """# Turing Machine M1
# the language of twin strings
# w&w   w in {0,1}*

start: q1
accept: A
reject: R

state: q1
    1 x r q3
    & & r q8
    0 x r q2

state: q2
    0 0 r q2
    1 1 r q2
    & & r q4

state: q3
    0 0 r q3
    1 1 r q3
    & & r q5

state: q4
    x x r q4
    0 x l q6
    
state: q5
    x x r q5
    1 x l q6

state: q6
    0 0 l q6
    1 1 l q6
    x x l q6
    & & l q7
    
state: q7
    0 0 l q7
    1 1 l q7
    x x r q1
    
state: q8
    x x r q8
    _ _ r A
    
"""


# Turing Machine M2, Sipser 3ird ed page 172, Sipser 2nd ed page 144
tm_M2 = """# Turing Machine M2
# the language consisting of all strings of 0s whose length 
# is a power of 2, A = { 0^2^n | n >=0 }

start: q1
accept: A
reject: R

state: q1
    0 _ r q2

state: q2
    x x r q2
    0 x r q3
    _ _ r A
    
state: q3
    x x r q3
    0 0 r q4
    _ _ l q5
    
state: q4
    x x r q4
    0 x r q3
    
state: q5
    0 0 l q5
    x x l q5
    _ _ r q2

"""


# Turing Machine M3, Sipser 3ird ed page 174, Sipser 2nd ed page 146
tm_M3 = """# Turing Machine M3
# The language of multiplication
# a^i b^j c^k, i,j,k >=1, and k = i*j

start: q0
accept: A
reject: R

state: q0
    a x r q1
    b b r q5

state: q1
    a a r q1
    b y r q2
    z z l q4

state: q2
    z z r q2
    b b r q2
    c z l q3

state: q3
    z z l q3
    b b l q3
    y y r q1
    
state: q4
    a a l q4
    y b l q4
    x x r q0

state: q5
    z z r q5
    b b r q5
    _ _ l A

"""


### Basic Test for Exercise A

In [62]:
   
exercise_A = [
    ('M1', tm_M1, (["&","10&10","000&000","10101&10101"],["000&001","00&000","000&00","0&","&0","110&111","00&1"])),
    ('M2', tm_M2, (["0","00","0000","00000000","0000000000000000"],["000","00000","0000000","0000000000"])),
    ('M3', tm_M3, (["abc","aaaabcccc","abbbbcccc","aabbcccc","aaabccc","aabbbcccccc"],["bac","aabbbccc","aabb","ca","abbccc","cbaa"])),
]


run_exercise(exercise_A)


Testing M1
accept:	|&|
accept:	|10&10|
accept:	|000&000|
accept:	|10101&10101|
reject:	|000&001|
reject:	|00&000|
reject:	|000&00|
reject:	|0&|
reject:	|&0|
reject:	|110&111|
reject:	|00&1|
correct: 11, incorrect: 0, exceptions: 0

Testing M2
accept:	|0|
accept:	|00|
accept:	|0000|
accept:	|00000000|
accept:	|0000000000000000|
reject:	|000|
reject:	|00000|
reject:	|0000000|
reject:	|0000000000|
correct: 9, incorrect: 0, exceptions: 0

Testing M3
accept:	|abc|
accept:	|aaaabcccc|
accept:	|abbbbcccc|
accept:	|aabbcccc|
accept:	|aaabccc|
accept:	|aabbbcccccc|
reject:	|bac|
reject:	|aabbbccc|
reject:	|aabb|
reject:	|ca|
reject:	|abbccc|
reject:	|cbaa|
correct: 12, incorrect: 0, exceptions: 0

correct: 3 out of 3 machines tested


True

### Exercise B

Give the Turing Machines that decide the following languages over the alphabet {0,1}.

a. {w | w contains an equal number of 0s and 1s}

b. {w | w contains twice as many 0s as 1s}

c. {w | w does not contain twice as many 0s as 1s}

This is exercise 3.8 from the class text, page 188 3ird edition, page 160 2nd edition.


In [70]:
# Exercise 3.8, Sipser


tm_3_8a = """# Exercise 3.8(a)
# the string w contains an equal number of 0s and 1s

accept: A
reject: R
start: q0

state: q0
    0 y r q1
    1 y r q2
    _ _ r A
    
state: q1
    0 0 r q1
    x x r q1
    1 x l q3
    
state: q2
    1 1 r q2
    x x r q2
    0 x l q3
    
state: q3
    0 0 l q3
    1 1 l q3
    x x l q3
    y y r q4
    
state: q4
    x x r q4
    _ _ r A
    0 x r q1
    1 x r q2
    

"""

tm_3_8b = """# Exercise 3.8(b)
# the string w contains twice as many 0s as 1s

accept: A
reject: R
start: q0

state: q0
    0 y r q1
    1 y r q2
    _ _ r A
    
state: q1
    0 0 r q1
    x x r q1
    1 x l q5
    
state: q2
    1 1 r q2
    x x r q2
    0 x l q5
    
state: q3
    0 0 l q3
    1 1 l q3
    x x l q3
    y y r q4
    
state: q4
    x x r q4
    _ _ r A
    0 x r q1
    1 x r q2
    
state: q5
    x x r q5
    1 1 r q5
    0 x l q3
    y y r q5

"""

tm_3_8c = """# Exercise 3.8(c)
# the string w does not contain twice as many 0s as 1s

accept: A
reject: R
start: q0

state: q0
    0 y r q1
    1 y r q2
    
state: q1
    0 0 r q1
    x x r q1
    1 x l q5
    _ _ r A
    
state: q2
    1 1 r q2
    x x r q2
    0 x l q5
    _ _ r A
    
state: q3
    0 0 l q3
    1 1 l q3
    x x l q3
    y y r q4
    _ _ r A
    
state: q4
    x x r q4
    _ _ r R
    0 x r q1
    1 x r q2
    
state: q5
    x x r q5
    1 1 r q5
    0 x l q3
    y y r q5
    _ _ r A

"""


### Basic Test for Exercise B

In [73]:
   
exercise_B = [
    ('3.8(a)', tm_3_8a, (["","01","0101","0011011100","001011","001110","01010110"],["1","00","101","0001111","010111","0001011"])),
    ('3.8(b)', tm_3_8b, (["","001","010","100001","100","110000","100010","111000000"],["0","1","10","110","001111","01010","0111000"])),
    ('3.8(c)', tm_3_8c, (["0","1","10","110","001111","01010","0111000","0010101"],["","001","010","001100","100001","100","110000","100010","111000000"])),
]

run_exercise(exercise_B)


Testing 3.8(a)
accept:	||
accept:	|01|
accept:	|0101|
accept:	|0011011100|
accept:	|001011|
accept:	|001110|
accept:	|01010110|
reject:	|1|
reject:	|00|
reject:	|101|
reject:	|0001111|
reject:	|010111|
reject:	|0001011|
correct: 13, incorrect: 0, exceptions: 0

Testing 3.8(b)
accept:	||
accept:	|001|
accept:	|010|
accept:	|100001|
accept:	|100|
accept:	|110000|
accept:	|100010|
accept:	|111000000|
reject:	|0|
reject:	|1|
reject:	|10|
reject:	|110|
reject:	|001111|
reject:	|01010|
reject:	|0111000|
correct: 15, incorrect: 0, exceptions: 0

Testing 3.8(c)
accept:	|0|
accept:	|1|
accept:	|10|
accept:	|110|
accept:	|001111|
accept:	|01010|
accept:	|0111000|
accept:	|0010101|
reject:	||
reject:	|001|
reject:	|010|
reject:	|001100|
reject:	|100001|
reject:	|100|
reject:	|110000|
reject:	|100010|
reject:	|111000000|
correct: 17, incorrect: 0, exceptions: 0

correct: 3 out of 3 machines tested


True

### Exercise C: Extra Credit

Program M4, the element distinctness problem, class text page 175 3ird edition, page 147 2nd edition.

In [8]:


tm_M4 = """# Turing Machine M4
# the element distinctness problem,
# E= { &x1&x2&...&xl | each xi in {0,1}* and xi != xj when i != j }

accept: A
reject: R
start: q0

state: q0      # endless loop. cause an exception
    : : n q0

"""


### Basic Test for Exercise C

In [9]:
   
exercise_C = [
    ('M4', tm_M4, (['&0&1','&0&00','&0&1&01'],["&0&0","&01&1&01","&0&11&11"])),
]

run_exercise(exercise_C)


Testing M4
exception:	|&0&1|
exception:	|&0&00|
exception:	|&0&1&01|
exception:	|&0&0|
exception:	|&01&1&01|
exception:	|&0&11&11|
correct: 0, incorrect: 0, exceptions: 6

correct: 0 out of 1 machines tested


False

### End of assigment
